In [2]:
"""
Converts from cartesian (x,y) to polar (r,θ) coordinates.
"""
function car2pol(x, y)
    r = √(x^2 + y^2)
    θ = atan(y, x)
    return r, θ
end;

"""
Wraps an angle (in radians) to the range [-π, π].
"""
function wrap2pi(θ)
    return θ - 2*π*floor((θ+π)/(2*π))
end;

"""
Converts from a vector of vectors to a matrix.
"""
function vec2mat(X::Vector{Vector{Float64}})::Matrix
    # convert a vector of vectors to a matrix 
    Xm = hcat(X...)
    return Xm 
end;

"""
Converts from a matrix to a vector of vectors.
"""
function mat2vec(Xm::Matrix)::Vector{Vector{Float64}}
    # convert a matrix into a vector of vectors 
    X = [Xm[:,i] for i = 1:size(Xm,2)]
    return X 
end;

A linear quadratic regulator (LQR) will be used to control the roll motion. The first step is thus to make a linearized system of the roll motion about an equilibrium point, which is selected to be the desired heel angle. The linearized system can be written as:

$$ x = \begin{bmatrix} \phi \\ p \end{bmatrix} (64) $$

$$\dot{x} = Ax + Bu. (65)$$

The restoring moment when linearised becomes: 
$$ G_{\phi} = \rho_w g \nabla GM_t((1 - 2\sin(\phi_w))^2) \phi + C_1 $$

where $\rho_w$ is sea water density, g is gravity, $\nabla$ is the total displacement of the ship, $GM_t$ is the transerse metacentric heigh, $ \phi_w $ is the equilibrium point 

and $C_1$ is equal to

$$ C_1 = \rho_w g \nabla GM_t\cos(\phi_w)\sin(\phi_w) $$

Making C1 and the moment generated by the keel a part of our control input u linearizes the system, and the linearized system model do then only includes the restoring moment. Matrices A and B in our linearized system then becomes:

$$
A = \begin{bmatrix}
0 & 1 \\
\frac{-\rho_w g \nabla GM_t(1 - 2\sin(\phi_w))^2}{I_{xx} - K_{\dot{p}}} & 0
\end{bmatrix} 

B = \begin{bmatrix}
0 \\
\frac{1}{I_{xx} - K_{\dot{p}}}
\end{bmatrix},
$$
where u is defined as
$$
u = -K \begin{bmatrix}
\phi - \phi_d \\
p
\end{bmatrix}
- K_i \int_0^t \phi - \phi_d d\tau + C_1 + F_{uh}z. 
$$

In [13]:
params = (
        m = 1600 # kg
        ρ = 1020 # kg/m3
        g = 9.81 # m/s2
        GMₜ = 2.4 # m
        Ixx = 6.8e3 # kg m2
        Kₚ = -1.0e3 # kg m2
        ∇ = m/ρ # m3
        h2 = 0.95 # m
        A = 1  # m^2, sail area
        l1 = 1  # m, distance from the center of mass to the sail
        l2 = 1  # m, distance from the center of mass to the rudder
        h = 1  # m, height of the sail
        )

0.95

In [10]:
Q = [0.75 0; 0 1]
R = 1e-10
A = [0 1; 0 0]

ϕw = x[1] # x is the state

A[1,2] = -ρ * g * ∇ * GMₜ * (1 - 2 * sin(ϕw))^2 / (Ixx - Kₚ)

B = [0; 1/(Ixx - Kp)]

UndefVarError: UndefVarError: x not defined

In [6]:
function FHLQR_solver(A::Matrix, B::Matrix, Q::Matrix, R::Matrix, Qf::Matrix, N::Int64)::Tuple{Vector{Matrix{Float64}}, Vector{Matrix{Float64}}}
    """
    Solves the continuous-time LQR problem.
    """

    nx,nu = size(B)
    @assert size(A) == (nx, nx)
    @assert size(Q) == (nx, nx)
    @assert size(R) == (nu, nu)
    @assert size(Qf) == (nx, nx)
    
    # instantiate S and K
    P = [zeros(nx,nx) for i = 1:N]
    K = [zeros(nu,nx) for i = 1:N-1]
    # initialize S[N] with Qf
    P[N] = deepcopy(Qf)

    for k = N:-1:2
        K[k-1] = (R + B'*P[k]*B)\(B'*P[k]*A)
        P[k-1] = Q + (A-B*K[k-1])'*P[k]*(A-B*K[k-1]) + K[k-1]'*R*K[k-1]
    end
    return K, P
end

function C1(ϕw)
    C1 = ρ * g * ∇ * GMₜ * cos(ϕw) * sin(ϕw)
    return C1
end

function undesirable_force_simple(Sy, Ry)
    """
    Simpler equation for the undesirable force.
    Taken from https://doi.org/10.1016/j.ifacol.2016.10.490 
    """
    # Sx = Force from sail in x direction
    # Ry = Force from rudder in y direction

    return -Sy - Ry
end

function undesirable_force(x, Sx, Sy, Rx, Ry)
    """
    ϕ = x[1]
    mu = 0
    nu = 0
    β = arctan2(nu * cos(ϕ), mu) # drift angle
    Sx = 0 # Force from sail in x direction
    Sy = 0 # Force from sail in y direction
    Rx = 0 # Force from rudder in x direction
    Ry = 0 # Force from rudder in y direction
    """
    
    

FHLQR_solver (generic function with 1 method)